In [19]:
import pandas as pd
import numpy as np

print("Đang tải dữ liệu train.csv và test.csv...")
try:
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file train.csv hoặc test.csv.")

test_passenger_ids = test_df['PassengerId']
y_train = train_df['Survived'] 

train_df = train_df.drop(['PassengerId', 'Survived'], axis=1)
test_df = test_df.drop('PassengerId', axis=1)

n_train = len(train_df) 
combined_df = pd.concat([train_df, test_df], axis=0, sort=False)
print(f"Đã gộp train ({n_train} dòng) và test ({len(test_df)} dòng).")

print("Bắt đầu áp dụng logic tiền xử lý...")


combined_df['HasCabin'] = combined_df['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)


combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
combined_df['Title'] = combined_df['Title'].replace('Mlle', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Ms', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Mme', 'Mrs')


combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1

combined_df['FamilySize_Grouped'] = 'Small' # Mặc định là 'Small' (cho 2, 3, 4)
combined_df.loc[combined_df['FamilySize'] == 1, 'FamilySize_Grouped'] = 'Alone'
combined_df.loc[combined_df['FamilySize'] >= 5, 'FamilySize_Grouped'] = 'Large'
print("Đã tạo feature mới: FamilySize_Grouped")


embarked_mode = combined_df['Embarked'].mode()[0]
combined_df['Embarked'] = combined_df['Embarked'].fillna(embarked_mode)
fare_median = combined_df['Fare'].median()
combined_df['Fare'] = combined_df['Fare'].fillna(fare_median)
age_median = combined_df['Age'].median()
combined_df['Age'] = combined_df['Age'].fillna(age_median)


combined_df['Age'] = pd.cut(combined_df['Age'], 
                            bins=[0, 16, 32, 48, 64, np.inf], 
                            labels=[0, 1, 2, 3, 4], 
                            right=False)


combined_df['Fare'] = pd.qcut(combined_df['Fare'], 4, labels=[0, 1, 2, 3])


combined_df = pd.get_dummies(combined_df, columns=['Sex', 'Title', 'Embarked', 'FamilySize_Grouped'], drop_first=True)


drop_cols = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'FamilySize']
combined_df = combined_df.drop(drop_cols, axis=1)

X_train_processed = combined_df[:n_train]
X_test_processed = combined_df[n_train:]

print("\n--- Tiền xử lý Hoàn tất (Đã thêm FamilySize_Grouped) ---")
print(f"Kích thước X_train: {X_train_processed.shape}")
print("\nCác cột mới bao gồm 'FamilySize_Grouped_Large', 'FamilySize_Grouped_Small':")
print(X_train_processed.head())

Đang tải dữ liệu train.csv và test.csv...
Đã gộp train (891 dòng) và test (418 dòng).
Bắt đầu áp dụng logic tiền xử lý...
Đã tạo feature mới: FamilySize_Grouped

--- Tiền xử lý Hoàn tất (Đã thêm FamilySize_Grouped) ---
Kích thước X_train: (891, 13)

Các cột mới bao gồm 'FamilySize_Grouped_Large', 'FamilySize_Grouped_Small':
   Pclass Age Fare  HasCabin  Sex_male  Title_Miss  Title_Mr  Title_Mrs  \
0       3   1    0         0      True       False      True      False   
1       1   2    3         1     False       False     False       True   
2       3   1    1         0     False        True     False      False   
3       1   2    3         1     False       False     False       True   
4       3   2    1         0      True       False      True      False   

   Title_Rare  Embarked_Q  Embarked_S  FamilySize_Grouped_Large  \
0       False       False        True                     False   
1       False       False       False                     False   
2       False       Fa

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import joblib

print("\n--- Bắt đầu Tinh chỉnh Mô hình (với feature FamilySize_Grouped) ---")

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [5, 10, 15, 20]
}
tree_model = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(estimator=tree_model, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1,
                           verbose=1)

print("Đang tìm tham số tốt nhất...")
grid_search.fit(X_train_processed, y_train) # Huấn luyện trên data mới

print(f"\nTham số tốt nhất tìm được: {grid_search.best_params_}")
print(f"Điểm accuracy (CV) tốt nhất (với FamilySize_Grouped): {grid_search.best_score_:.4f}")

best_tree_model = grid_search.best_estimator_


print("\n--- Đang dự đoán và xuất file submission ---")

y_test_pred = best_tree_model.predict(X_test_processed) # Dự đoán trên data mới

submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': y_test_pred
})


submission_filename = 'submission.csv' 
submission.to_csv(submission_filename, index=False)
print(f"Tạo file '{submission_filename}' thành công!")


--- Bắt đầu Tinh chỉnh Mô hình (với feature FamilySize_Grouped) ---
Đang tìm tham số tốt nhất...
Fitting 5 folds for each of 192 candidates, totalling 960 fits

Tham số tốt nhất tìm được: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 5}
Điểm accuracy (CV) tốt nhất (với FamilySize_Grouped): 0.8305

--- Đang dự đoán và xuất file submission ---
Tạo file 'submission.csv' thành công!
